In [7]:
# !pip3 install  torchaudio librosa 

In [8]:
from  transformers import  *
import  torch
import  torchaudio
import  librosa
import  numpy  as  np
import  pandas  as  pd
import  matplotlib.pyplot  as  plt
import  IPython.display  as  ipd
import  os


/opt/python/3.10.4/lib/python3.10/site-packages/torchaudio/_internal/module_utils.py:99: UserWarning: Failed to import soundfile. 'soundfile' backend is not available.
  warnings.warn("Failed to import soundfile. 'soundfile' backend is not available.")


OSError: sndfile library not found

In [ ]:
# enocode  audion file  to  features
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Downloading: 100%|██████████| 163/163 [00:00<00:00, 79.9kB/s]
Downloading: 100%|██████████| 85.0/85.0 [00:00<00:00, 40.1kB/s]
loading file vocab.json from cache at /home/codespace/.cache/huggingface/hub/models--facebook--wav2vec2-base-960h/snapshots/706111756296bc76512407a11e69526cf4e22aae/vocab.json
loading file tokenizer_config.json from cache at /home/codespace/.cache/huggingface/hub/models--facebook--wav2vec2-base-960h/snapshots/706111756296bc76512407a11e69526cf4e22aae/tokenizer_config.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /home/codespace/.cache/huggingface/hub/models--facebook--wav2vec2-base-960h/snapshots/706111756296bc76512407a11e69526cf4e22aae/special_tokens_map.json
Downloading: 100%|██████████| 1.60k/1.60k [00:00<00:00, 629kB/s]
loading configuration file config.json from cache at /home/codespace/.cache/huggingface/hub/models--facebook--wav2vec2-base-960h/snapshots/706111756296bc76512407a11e69526cf4e22aae/con

In [14]:
audio='./141.wav'
#mono
audio_signal,sr=torchaudio.load(audio,channels_first=False)
len(audio_signal),sr

(674816, 44100)